<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/FinalPrescription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Prescription.csv')
df.head()

,PatientUID,PatientVisitUID,DispensedDttm,DrugName,GenericName,Quantity,DrugFrequency,Dosage,DosageForm,DrugDuration
0,2299,2024049,2558-02-19 14:48:15.693,Folic Acid 5 mg TAB,Folivit tab 5 mg,90.0,od pc |วันละ 1 ครั้ง หลังอาหารเช้า,1.0,1 TAB,90.0
1,2299,2024049,2558-02-19 14:48:15.693,Vitamin B Complex TAB,Vitamin B Complex TAB,180.0,bid pc |วันละ 2 ครั้ง หลังอาหารเช้า เย็น,1.0,1 TAB,90.0
2,2299,2024049,2558-02-19 14:48:15.693,losartan(Loranta) 50 mg TAB,Loranta film-coated tab 50 mg,90.0,od pc |วันละ 1 ครั้ง หลังอาหารเช้า,1.0,1 TAB,90.0
3,2299,2024049,2558-02-19 14:48:15.693,lorazepam(Anxira) 0.5 mg TAB,Anxira tab 0.5 mg,30.0,hs |วันละ 1 ครั้ง ก่อนนอน,1.0,1 TAB,30.0
4,2299,2024049,2558-02-19 14:47:55.953,ChalkTAB-1.5 TAB,Chalktab 1500 mg,90.0,od pc |วันละ 1 ครั้ง หลังอาหารเช้า,1.0,1 TAB,90.0


In [35]:
import pandas as pd
import re

# Lowercase and store values from 'DrugName' column
drugCol1 = [str(i).lower() for i in df['DrugName']]

# Lowercase and store values from 'GenericName' column
genericCol1 = [str(i).lower() for i in df['GenericName']]

# Store drug names
drugCol2 = []
genericCol2 = []

# Process 'DrugName' column
for value in drugCol1:
    if isinstance(value, str):
        if value[0].isdigit():
            drugCol2.append(value)
        else:
            valList = value.split()
            index_begin_with_digit = len(valList)
            for i in range(len(valList)):
                if valList[i][0].isdigit():
                    index_begin_with_digit = i
                    break
            newval = " ".join(valList[:index_begin_with_digit])
            drugCol2.append(newval)

# Process 'GenericName' column
for value in genericCol1:
    if isinstance(value, str):
        if value[0].isdigit():
            genericCol2.append(value)
        else:
            valList = value.split()
            index_begin_with_digit = len(valList)
            for i in range(len(valList)):
                if valList[i][0].isdigit():
                    index_begin_with_digit = i
                    break
            newval = " ".join(valList[:index_begin_with_digit])
            genericCol2.append(newval)

# Replace unwanted words and remove words starting with 'no.' and the digits following them
words_to_remove = ['inj', 'cap', 'mg', 'ml', 'tab', '[7402]', '[5702]', '[5703]', '[0604]', '[5606]', '[024840]', '[napha]', '[5704]', '(ฝาก)', '(เลิกใช้)']

drugCol3 = []
for i in drugCol2:
    for word in words_to_remove:
        i = i.replace(word, "")
    i = re.sub(r'\bno\.\d+(\.\d+)?\b', '', i)
    drugCol3.append(i.strip())

genericCol3 = []
for i in genericCol2:
    for word in words_to_remove:
        i = i.replace(word, "")
    i = re.sub(r'\bno\.\d+(\.\d+)?\b', '', i)
    genericCol3.append(i.strip())

# Print the updated lists
#print("Updated drugCol3:", drugCol3)
#print("Updated genericCol3:", genericCol3)

In [36]:
# Create DataFrame with 'DrugName' and 'GenericName' columns
df_combined = pd.DataFrame({'DrugName': drugCol3, 'GenericName': genericCol3})

# Count unique values for 'DrugName'
drug_value_counts = df_combined['DrugName'].value_counts()

# Count unique values for 'GenericName'
generic_value_counts = df_combined['GenericName'].value_counts()

In [37]:
df_combined

,DrugName,GenericName
0,folic acid,folivit
1,vitamin b complex,vitamin b complex
2,losartan(loranta),loranta film-coated
3,lorazepam(anxira),anxira
4,chalk-1.5,chalk
...,...,...
9530447,infusion plabottle set (iv.set),nan
9530448,normal saline,normal saline
9530449,normal saline,normal saline
9530450,minimun volume extension with t connector,nan


In [38]:
pre_drug = df_combined['DrugName'].value_counts().reset_index()
pre_drug.columns = ['DrugName', 'Count']

pre_generic = df_combined['GenericName'].value_counts().reset_index()
pre_generic.columns = ['GenericName', 'Count']

pre_count = df_combined.groupby(['DrugName', 'GenericName']).size().reset_index()
pre_count.columns = ['DrugName', 'GenericName', 'Count']

In [39]:
pre_df = pd.DataFrame(pre_count)
pre_drug = pd.DataFrame(pre_drug)
pre_generic = pd.DataFrame(pre_generic)

In [40]:
with pd.ExcelWriter('PrescriptionCount.xlsx') as writer:
    pre_df.to_excel(writer, sheet_name='Prescription')
    pre_drug.to_excel(writer, sheet_name='CleanDrugName')
    pre_generic.to_excel(writer, sheet_name='CleanGenericName')

In [41]:
pre_count

,DrugName,GenericName,Count
0,2% chlorhexidine in 70% alcohol 450,2% chlorhexidine in 70% alcohol 500,47
1,3tc 150,3tc 150,800
2,3tc 300,lamivir,289
3,3tc lamivudine 150,lamivudine mylan film-coated,10751
4,3tc suspension,3tc suspension,1
...,...,...,...
4645,ไหมเย็บเนื้อเยื่อเข้ากับสมอยึดตรีงกระดูกultrab...,nan,1
4646,ไหมเย็บแผล ชนิดเส้นเดี่ยว ไม่ละลายน้ำ ผลิตจากพ...,nan,1
4647,ไหมเย็บแผล ชนิดเส้นเดี่ยว ไม่ละลายน้ำ ผลิตจากโ...,nan,1
4648,ไหมเย็บแผล ชนิดเส้นเดี่ยว ไม่ละลายน้ำ ผลิตจากโ...,nan,1


In [42]:
# Calculate the sum of values in the 'Count' column
count_sum = pre_count['Count'].sum()

# Print the sum
print("Sum of 'Count' column:", count_sum)

Sum of 'Count' column: 9530452
